# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откроим файл с данными и изучим общую информацию

In [1]:
import pandas as pd                          # импорт библиотеки pandas
df = pd.read_csv('data.csv')       # чтение файла с данными и сохранение в df
df.head(10)                                  # получение первых 10 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
df.info()   # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В каждой строке таблицы — данные потенциального заёмщика. Часть колонок описывает заёмщика: пол, возраст, стаж, семейное положение, образование, колчество детей, тип занятости, задолженность и цель получения кредита . Остальные данные: индификаторы уровня образвания и семейного положения. 

Предварительно можно утверждать, что, данных достаточно для иследования. Но встречаются пропуски в данных, различные типы данных, отрицательные числа в колонках, строчные буквы сочетаются с прописными.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
(df.isna().mean()*100).round() # подсчёт пропусков в процентах

children             0.0
days_employed       10.0
dob_years            0.0
education            0.0
education_id         0.0
family_status        0.0
family_status_id     0.0
gender               0.0
income_type          0.0
debt                 0.0
total_income        10.0
purpose              0.0
dtype: float64

In [4]:
df[df['dob_years'] == 0].head(10) # проверка колонки на зачения =0

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
1149,0,-934.654854,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости
1175,0,370879.508002,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1386,0,-5043.219890,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем
1890,0,NaN,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,NaN,жилье
1898,0,370144.537021,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля


In [5]:
df['dob_years'] = df['dob_years'].replace(0, df['dob_years'].median()) # замена нулевых значений на медианные в колнке возраста

df.reset_index(drop=True)
df['education'] = df['education'].str.lower() 


In [4]:
df.groupby(['income_type', 'education']).agg({'total_income':'median'})

total_income
income_type     education                         
безработный     Высшее               202722.511368
                среднее               59956.991984
в декрете       СРЕДНЕЕ               53829.130729
госслужащий     ВЫСШЕЕ               174909.179988
                Высшее               161671.347625
                Начальное            190912.178349
                Неоконченное высшее  107224.860090
                СРЕДНЕЕ              132756.053048
                Среднее              148529.629601
                высшее               172511.107016
                начальное            105766.403301
                неоконченное высшее  165106.669771
                среднее              136652.970357
                ученая степень       111392.231107
компаньон       ВЫСШЕЕ               200701.997023
                Высшее               192451.667511
                НАЧАЛЬНОЕ            190952.908390
                НЕОКОНЧЕННОЕ ВЫСШЕЕ  145528.679914
                Начальное             88227.830722
                Неоконченное высшее  183379.320368
                СРЕДНЕЕ              157308.365600
                Среднее              149190.998579
                высшее               201911.716329
                начальное            170077.891115
                неоконченное высшее  179867.152890
                среднее              159652.900458
пенсионер       ВЫСШЕЕ               165563.164062
                Высшее               154962.528710
                НАЧАЛЬНОЕ             90479.336389
                НЕОКОНЧЕННОЕ ВЫСШЕЕ  412386.347106
                Начальное             98853.333449
                Неоконченное высшее   83625.583859
                СРЕДНЕЕ              113656.027279
                Среднее              128155.229215
                Ученая степень        98752.495442
                высшее               142825.046222
                начальное            104505.757848
                неоконченное высшее  122706.716059
                среднее              114483.373934
                ученая степень       255425.196556
предприниматель высшее               499163.144947
сотрудник       ВЫСШЕЕ               144683.271465
                Высшее               178238.955336
                НАЧАЛЬНОЕ            212545.366151
                НЕОКОНЧЕННОЕ ВЫСШЕЕ  166015.790504
                Начальное            129895.001992
                Неоконченное высшее  159115.726175
                СРЕДНЕЕ              135823.934197
                Среднее              133399.107243
                УЧЕНАЯ СТЕПЕНЬ       198570.757322
                высшее               166164.078024
                начальное            124298.759469
                неоконченное высшее  149551.191876
                среднее              136818.115423
                ученая степень       192180.127162
студент         высшее                98201.625314

In [8]:
qq=df.pivot_table(index=['income_type', 'gender'], columns='education', values='total_income', aggfunc='median')
qq

education                      ВЫСШЕЕ         Высшее      НАЧАЛЬНОЕ  \
income_type     gender                                                
безработный     F                 NaN  202722.511368            NaN   
                M                 NaN            NaN            NaN   
в декрете       F                 NaN            NaN            NaN   
госслужащий     F       174622.608484  161671.347625            NaN   
                M       207561.466998  245357.105783            NaN   
компаньон       F       195156.487586  190715.230845  255696.018913   
                M       205309.210389  215733.128065  131455.738975   
                XNA               NaN            NaN            NaN   
пенсионер       F       164824.382522  153869.596567   70278.364094   
                M       178437.573825  362784.960182  104293.022877   
предприниматель F                 NaN            NaN            NaN   
сотрудник       F       142495.413236  170693.563511  190146.365776   
                M       170788.265697  196164.472215  236593.643905   
студент         M                 NaN            NaN            NaN   

education               НЕОКОНЧЕННОЕ ВЫСШЕЕ      Начальное  \
income_type     gender                                       
безработный     F                       NaN            NaN   
                M                       NaN            NaN   
в декрете       F                       NaN            NaN   
госслужащий     F                       NaN            NaN   
                M                       NaN  190912.178349   
компаньон       F             126061.010002   83687.606219   
                M             188626.957194  116365.818367   
                XNA                     NaN            NaN   
пенсионер       F             412386.347106   90274.492041   
                M                       NaN  101424.984669   
предприниматель F                       NaN            NaN   
сотрудник       F             166015.790504  102669.099092   
                M             193194.767463  159584.822691   
студент         M                       NaN            NaN   

education               Неоконченное высшее        СРЕДНЕЕ        Среднее  \
income_type     gender                                                      
безработный     F                       NaN            NaN            NaN   
                M                       NaN            NaN            NaN   
в декрете       F                       NaN   53829.130729            NaN   
госслужащий     F              94288.056759  124586.373803  130000.920706   
                M             133110.642705  136080.408092  177546.333151   
компаньон       F             182170.123960  149535.147678  132967.088517   
                M             184588.516776  184625.570681  185734.374253   
                XNA                     NaN            NaN            NaN   
пенсионер       F              83625.583859  112080.767987  130275.002434   
                M                       NaN  128882.579913  115649.508864   
предприниматель F                       NaN            NaN            NaN   
сотрудник       F             149788.794193  113004.887539  117497.925183   
                M             167791.985138  168756.416771  156451.271417   
студент         M                       NaN            NaN            NaN   

education               УЧЕНАЯ СТЕПЕНЬ  Ученая степень         высшее  \
income_type     gender                                                  
безработный     F                  NaN             NaN            NaN   
                M                  NaN             NaN            NaN   
в декрете       F                  NaN             NaN            NaN   
госслужащий     F                  NaN             NaN  156817.746136   
                M                  NaN             NaN  217201.948077   
компаньон       F                  NaN             NaN  189772.345890   
                M                  NaN             Na

In [7]:
def super_fillna_func(income_type, gender, education):
    '''
    Находит в таблице qq нужную медиану.
    '''
    try:
        return qq.loc[(income_type, gender)][education]
    except:
        return 0
    
print(super_fillna_func('студент', 'M','высшее') )

98201.62531401133


In [9]:
df.apply(lambda row: super_fillna_func(row['income_type'], row['gender'], row['education']), axis=1)

0        153478.301459
1        124665.734120
2        156451.271417
3        155791.706825
4        112363.944049
             ...      
21520    144902.645557
21521    112363.944049
21522    155791.706825
21523    155791.706825
21524    124665.734120
Length: 21525, dtype: float64

In [10]:
df['total_income'] = df['total_income'].fillna(df.apply(lambda row: super_fillna_func(row['income_type'], row['gender'], row['education']), axis=1))
df['days_employed'] = df['days_employed'].median()
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Обнаружены пропуски в столбцах 'days_employed' и 'total_income', заменены все пропуски данных столбцов на медианные значения.

### Замена типа данных

In [26]:
df['days_employed'] = df['days_employed'].apply(abs) # отрицательные значения в 'days_employed' делаем положительными
df['children'] = df['children'].apply(abs) # отрицательные значения в 'children' делаем положительными

df['total_income'] = df['total_income'].astype('int') # меняем тип данных на 'int'

df['days_employed'] = df['days_employed'].astype('int') # меняем тип данных на 'int'

df.dtypes # получение информации о типах данных



children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

**Вывод**

При выводе общей информации таблицы были обнаружены вещественные типы данных в 'total_income' и  'days_employed', отрицательные значения в 'days_employed' и 'children'. Вещественные типы данных заменены на целые и отрицательные значения заменены на положительные.

### Обработка дубликатов

In [27]:
duplicated_df = df[df.duplicated()].head() #просмотр явных дубликатов
duplicated_df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,1203,41,среднее,1,женат / замужем,0,F,сотрудник,0,124127,покупка жилья для семьи
3290,0,1203,58,среднее,1,гражданский брак,1,F,пенсионер,0,112790,сыграть свадьбу
4182,1,1203,34,высшее,0,гражданский брак,1,F,сотрудник,0,153691,свадьба
4851,0,1203,60,среднее,1,гражданский брак,1,F,пенсионер,0,112790,свадьба
5557,0,1203,58,среднее,1,гражданский брак,1,F,пенсионер,0,112790,сыграть свадьбу


In [28]:
print('Количество явных дубликатов:',df.duplicated().sum()) # количество явных дубликатов

Количество явных дубликатов: 72


In [29]:
df['education'] = df['education'].str.lower() # проблемы с регистром столбца 'education', исправляем

df = df.drop_duplicates().reset_index(drop=True) #удаляем явные дубликаты

print('Количество явных дубликатов:',df.duplicated().sum()) 

Количество явных дубликатов: 0


In [30]:
df['family_status'].value_counts() #просмотр неявных дубликатов столбца 'family_status'

женат / замужем          12339
гражданский брак          4150
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [31]:
#df = df.replace('гражданский брак', 'женат / замужем') # замена идентичных по смыслу значений
df = df.replace('Не женат / не замужем', 'холост') # замена идентичных по смыслу значений для лучшего восприятия

df.reset_index(drop=True)

df['family_status'].value_counts()

женат / замужем     12339
гражданский брак     4150
холост               2810
в разводе            1195
вдовец / вдова        959
Name: family_status, dtype: int64

In [32]:
df['gender'].value_counts() #просмотр неявных дубликатов столбца 'gender'

F      14173
M       7279
XNA        1
Name: gender, dtype: int64

In [33]:
df['income_type'].value_counts() #просмотр неявных дубликатов столбца 'income_type'

сотрудник          11083
компаньон           5078
пенсионер           3829
госслужащий         1457
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [34]:
df['purpose'].value_counts() #просмотр неявных дубликатов столбца 'purpose'

свадьба                                   791
на проведение свадьбы                     767
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

**Вывод**

Были обнаружены явные дубликаты, проблемы с регистром в 'education'. Удалены явные дубликаты и исправлен регистр.
При просмотре неявных дубликатов столбцов, были обнаружены дубликаты в 'purpose'.

### Лемматизация

In [35]:
from pymystem3 import Mystem 
m = Mystem()

In [36]:
def lemmatizer(df, column_name): # функция лемматизирует каждую цель, затем по ключевому слову заменяет цель в таблице
    for i in range(len(df[column_name])):
        lemmas = m.lemmatize(df.loc[i,column_name])
        if 'свадьба' in lemmas:
            df.loc[i,column_name] = 'свадьба'
        if 'жилье' in lemmas or 'недвижимость' in lemmas:
            df.loc[i,column_name] = 'недвижимость'
        if 'автомобиль' in lemmas:
            df.loc[i,column_name] = 'автомобиль'
        if 'образование' in lemmas:
            df.loc[i, column_name] = 'образование'
    return df

df = lemmatizer(df, 'purpose')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,1203,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,1203,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,1203,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,1203,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,1203,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
5,0,1203,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость
6,0,1203,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость
7,0,1203,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,1203,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба
9,0,1203,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость


**Вывод**

Удалили неявные дубликаты в 'purpose' лемматизацией, привели к обобщенным значениям.

### Категоризация данных

In [37]:
income_categor = df[['debt', 'income_type']]
worker = income_categor[(income_categor['income_type'] == 'сотрудник') &  # работающие заемщики
                        (income_categor['income_type'] == 'компаньон') &
                        (income_categor['income_type'] == 'госслужащий')]

pension = income_categor[income_categor['income_type'] == 'пенсионер']  # пенсионеры заемщики  

other = income_categor[(income_categor['income_type'] == 'безработный') & # другие заемщики
                       (income_categor['income_type'] == 'в декрете') &
                       (income_categor['income_type'] == 'студент') &
                       (income_categor['income_type'] == 'предприниматель')]
                    
work_conv = (worker[worker['debt'] == 1].count() / worker['debt'].count())[0]

pension_conv = (pension[pension['debt'] == 1].count() / pension['debt'].count())[0]

other_conv = (other[worker['debt'] == 1].count() / other['debt'].count())[0]

income_conv_df = {'работающие': [work_conv], 'пенсионеры' : [pension_conv], 'другие' : [other_conv]}
income_conversion_table = pd.DataFrame(data = income_conv_df) # таблица с коверсиями возвращения кредита по типу занятости, 
                                                              # пенсионеры чаще не возвращают кредиты
                                                          

print('Конверсия для работающих заемщиков: {:.2%}'.format(work_conv))
print('Конверсия для пенсионеров: {:.2%}'.format(pension_conv))
print('Конверсия для других заемщиков: {:.2%}'.format(other_conv))
income_conversion_table

df.groupby('income_type')['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])

Конверсия для работающих заемщиков: nan%
Конверсия для пенсионеров: 5.64%
Конверсия для других заемщиков: nan%


,count,sum,<lambda_0>
income_type,,,
безработный,2,1,50.00%
в декрете,1,1,100.00%
госслужащий,1457,86,5.90%
компаньон,5078,376,7.40%
пенсионер,3829,216,5.64%
предприниматель,2,0,0.00%
сотрудник,11083,1061,9.57%
студент,1,0,0.00%


**Вывод**

По колонке <тип занятости>  получается, что сотрудники чаще не возвращают кредиты, а пенсионеры наоборот,
самые примерные плательщики

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [39]:
df['children'].value_counts() # просмотр уникальных значений, у 76 заемщиков по 20 детей, видимо тоже опечатка

0     14090
1      4855
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [40]:
median_child=df[df['children'] != 20]['children'].median() # исправляем 20 детей, медианным значением
df['children'] = df['children'].replace(20, median_child)
df['children'].value_counts() # просмотр данных, данные готовы для анализа

0    14166
1     4855
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

In [41]:
child_categor = df[['debt', 'children']]

no_child = df[df['children'] == 0]['children'].count() # заемщики без детей
one_child = df[df['children'] == 1]['children'].count() # заемщики с 1 ребенком
two_and_more_child = df[df['children'] > 1]['children'].count() # заемщики имеющие 2 и более  детей

# ищем конверсии по заемщикам в зависимости от количества детей
no_child_conv = (child_categor[(child_categor['debt'] == 1) & (child_categor['children'] == 0)].count() / no_child)[0]
one_child_conv = (child_categor[(child_categor['debt'] == 1) & (child_categor['children'] == 1)].count() / one_child)[0]
two_and_more_child = (child_categor[(child_categor['debt'] == 1) &
                                    (child_categor['children'] > 1)].count() / two_and_more_child)[0]

# формируем дата фрейм по полученным данным
child_conv_df = {'no_child': [no_child_conv], 'one_child' : [one_child_conv], 'two_and_more_child' : [two_and_more_child]}
child_conversion_table = pd.DataFrame(data = child_conv_df)

print('Конверсия для заемщиков не имеющих детей : {:.2%}'.format(no_child_conv))
print('Конверсия для заемщиков имеющих одного ребенка : {:.2%}'.format(one_child_conv))
print('Конверсия для заемщиков имеющих более одного ребенка : {:.2%}'.format(two_and_more_child))
child_conversion_table

Конверсия для заемщиков не имеющих детей : 7.56%
Конверсия для заемщиков имеющих одного ребенка : 9.17%
Конверсия для заемщиков имеющих более одного ребенка : 9.25%


,no_child,one_child,two_and_more_child
0,0.075604,0.091658,0.092516


In [42]:
# то же самое, что и выше, только короче, ищем конверсию
child_conv = (df[df['debt'] == 1].groupby('children')['debt'].count() / df.groupby('children')['debt'].count())
child_conv.to_frame('conversion table')


,conversion table
children,
0,0.075604
1,0.091658
2,0.094542
3,0.081818
4,0.097561
5,NaN


In [43]:
df.groupby('children')['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])

,count,sum,<lambda_0>
children,,,
0,14166,1071,7.56%
1,4855,445,9.17%
2,2052,194,9.45%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.00%


In [44]:
df['children'].apply(lambda x: 'Есть' if x>0 else 'Нет')

0        Есть
1        Есть
2         Нет
3        Есть
4         Нет
         ... 
21448    Есть
21449     Нет
21450    Есть
21451    Есть
21452    Есть
Name: children, Length: 21453, dtype: object

In [11]:
df.groupby( df['children'].apply(lambda x: 'Есть' if x>0 else 'Нет') )['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])

,count,sum,<lambda_0>
children,,,
Есть,7329,677,9.24%
Нет,14196,1064,7.50%


**Вывод**

Поучается, что есть небольшая зависимость заемщиков от количества детей, заемщики без детей лучше отдают кредит.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [46]:
family_status_conv = (df[df['debt'] == 1].groupby('family_status')['debt'].count() / df.groupby('family_status')['debt'].count())
family_status_conv.to_frame('conversion table') #ищем конверсию

,conversion table
family_status,
в разводе,0.071130
вдовец / вдова,0.065693
гражданский брак,0.093494
женат / замужем,0.075452
холост,0.097509


In [47]:
df.groupby('family_status')['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])

,count,sum,<lambda_0>
family_status,,,
в разводе,1195,85,7.11%
вдовец / вдова,959,63,6.57%
гражданский брак,4150,388,9.35%
женат / замужем,12339,931,7.55%
холост,2810,274,9.75%


In [12]:
dict(zip(df['family_status_id'], df['family_status']))

{0: 'женат / замужем',
 1: 'гражданский брак',
 2: 'вдовец / вдова',
 3: 'в разводе',
 4: 'Не женат / не замужем'}

**Вывод**

Заемщики категорий вдовец / вдова и в разводе отдают кредит лучше.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [50]:
pd.qcut(df['total_income'], q=4) #делим на 4 равные группы по доходу методом qcut

0        (196904.0, 2265604.0]
1         (107499.0, 144881.0]
2         (144881.0, 196904.0]
3        (196904.0, 2265604.0]
4         (144881.0, 196904.0]
                 ...          
21448    (196904.0, 2265604.0]
21449     (144881.0, 196904.0]
21450       (-0.001, 107499.0]
21451    (196904.0, 2265604.0]
21452       (-0.001, 107499.0]
Name: total_income, Length: 21453, dtype: category
Categories (4, interval[float64]): [(-0.001, 107499.0] < (107499.0, 144881.0] < (144881.0, 196904.0] < (196904.0, 2265604.0]]

In [13]:
pd.qcut(df['total_income'], 5, ['низкий','ниже среднего','средний','выше среднего', 'высокий'])

0              высокий
1        ниже среднего
2              средний
3              высокий
4              средний
             ...      
21520          высокий
21521          средний
21522           низкий
21523          высокий
21524           низкий
Name: total_income, Length: 21525, dtype: category
Categories (5, object): ['низкий' < 'ниже среднего' < 'средний' < 'выше среднего' < 'высокий']

In [15]:
#  создаем функцию по уровню зароботка на основаниии 4-х групп, ищем конверсию в зависимости от функции по категориям  з/п
def cash_group(cash): 
    if cash <= 107623:
        return 'poor'
    if cash <= 145017:
        return 'aver'
    if cash <= 195813.25:
        return 'rich'
    return 'very_rich'

df['cash_group'] = df['total_income'].apply(cash_group)

cash_conv = (df[df['debt'] == 1].groupby('cash_group')['debt'].count() / df.groupby('cash_group')['debt'].count())
cash_conv.to_frame('conversion table')


,conversion table
cash_group,
aver,0.085230
poor,0.079471
rich,0.088224
very_rich,0.070865


In [16]:
df['cash_group'].value_counts()

aver         5491
very_rich    5447
poor         5373
rich         5214
Name: cash_group, dtype: int64

In [17]:
df.groupby('cash_group')['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])

,count,sum,<lambda_0>
cash_group,,,
aver,5491,468,8.52%
poor,5373,427,7.95%
rich,5214,460,8.82%
very_rich,5447,386,7.09%


**Вывод**

Зависимость есть, как ни странно, лучше всего отдают кредит заемщики у которых зарплата менее или равна 107623 и заемщики с зарплатой более 195813

- Как разные цели кредита влияют на его возврат в срок?

In [55]:
purpose_conv = (df[df['debt'] == 1].groupby('purpose')['debt'].count() / df.groupby('purpose')['debt'].count())
purpose_conv.to_frame('conversion table') #ищем конверсию


,conversion table
purpose,
автомобиль,0.093590
недвижимость,0.072334
образование,0.092200
свадьба,0.080069


In [56]:
df.groupby('purpose')['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])

,count,sum,<lambda_0>
purpose,,,
автомобиль,4306,403,9.36%
недвижимость,10811,782,7.23%
образование,4013,370,9.22%
свадьба,2323,186,8.01%


**Вывод**

Охотнее всего заемщики платят за кредит имеющий дело с недвижимостью.

## Шаг 4. Общий вывод

Исследование надежности заемщиков показало, что семейное положение и количество детей клиента влияет на факт погашения кредита в срок. 

Надежные клиенты - заемщики из категорий:
   
* пенсионер - 5.64% не возврата кредита в срок;
* не имеющие детей - 7.56% не возврата кредита в срок;
* семейное положение "вдовец / вдова" - 6.57% не возврата кредита в срок;
* с заработной платой  более 195813 - 7.14% не возврата кредита в срок;
* кредит на операции с недвижимостью - 7.23% не возврата кредита в срок.

Самые не надежные клиенты - заемщики из категорий:
 
* сотрудник - 9.57%  не возврата кредита в срок;
* имеющие одного или более детей - от 9.17%  до 9.76% не возврата кредита в срок;
* семейное положение "холост" - 9.75% не возврата кредита в срок;
* с заработной платой  более от 145017 до 195813 - от 8.53%  до 8.91% не возврата кредита в срок;
* кредит на операции с авто - 9.36% не возврата кредита в срок.